In [1]:
import yasa
import os
import mne
from glob import glob
from scipy import signal
from pprint import pprint
import joblib
import numpy as np

ModuleNotFoundError: No module named 'yasa'

In [4]:
!python3 -m pip install yasa


  Using cached yasa-0.6.3-py3-none-any.whl
  Using cached numba-0.58.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.7 kB)
  Using cached outdated-0.2.2-py2.py3-none-any.whl (7.5 kB)
  Using cached antropy-0.1.6-py3-none-any.whl
  Using cached scikit_learn-1.3.2-cp311-cp311-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached tensorpac-0.6.5-py3-none-any.whl (423 kB)
  Using cached pyriemann-0.5-py2.py3-none-any.whl
  Using cached sleepecg-0.5.5-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.3 kB)
  Using cached lspopt-1.3.0-py2.py3-none-any.whl (35 kB)
  Using cached ipywidgets-8.1.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached lightgbm-4.1.0.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached stochastic-0.6.0-py3-none-any.whl (49 kB)
  Using cached llvmlite-0.41.

In [72]:
def welch_psd(timeserie, fs):
    win = int(4 * fs)  # Window size is set to 4 seconds
    freqs, psd = signal.welch(timeserie, fs, nperseg=win, average='median')
    return freqs, psd
    
def band_power_features(timeserie, fs):
    freqs, psd = welch_psd(timeserie, fs)
    bp = yasa.bandpower_from_psd(psd, freqs).to_dict()
    del bp['Chan']
    del bp['Relative']
    del bp['TotalAbsPow']
    del bp['FreqRes']
    bp = {f"{k}_rel":v[0] for k,v in bp.items()}

    # when you want to get absolute power band uncomment below 
    # bp_abs = yasa.bandpower_from_psd(psd, freqs, relative=False).to_dict()
    # del bp_abs['Chan']
    # del bp_abs['Relative']
    # del bp_abs['TotalAbsPow']
    # del bp_abs['FreqRes']
    # bp_abs = {f"{k}_abs":v[0] for k,v in bp_abs.items()}
    # bp.update(bp_abs)

    return bp

def slider(sig):
    STEP = 4 * FS
    OVERLAP = 0.5
    for i, j in zip(range(0, len(sig), int(STEP*0.5)), range(STEP, len(sig), int(STEP*0.5))):
        yield sig[i:j]

In [1]:
FS = 500
path = r"/Applications/OPENneuro dataset 88/derivatives"
participants = pd.read_csv("/Applications/OPENneuro dataset 88/participants.tsv", sep="\t")

NameError: name 'pd' is not defined

In [105]:
participants.head(10)

,participant_id,Gender,Age,Group,MMSE
0,sub-001,F,57,A,16
1,sub-002,F,78,A,22
2,sub-003,M,70,A,14
3,sub-004,F,67,A,20
4,sub-005,M,70,A,22
5,sub-006,F,61,A,14
6,sub-007,F,79,A,20
7,sub-008,M,62,A,16
8,sub-009,F,77,A,23
9,sub-010,M,69,A,20


In [110]:
participants[participants.participant_id == "sub-006"].Group.values[0]

'A'

In [74]:
%%time
dirs = os.listdir(path)

notfound = list()
result = dict()

for d in dirs:
    tmp = dict()
    p = glob(os.path.join(path, d, "eeg", "*.set"))
    if len(p) == 0:
        notfound.append(d)
        print(f"eeg not found in: {d}")
        
    elif len(p) == 1:
        print(p[0])
        sig = mne.io.read_raw_eeglab(p[0])
        for s, c in zip(sig.get_data(), sig.ch_names):
            tmp_c = list()
            for win in slider(s):
                tmp_c.append(band_power_features(win, FS))
                
            tmp.update({
                c: tmp_c
            })
        result.update({d: tmp})
    else:
        raise FileExistsError(f"too many eeg found in subject: {d}")

     

eeg not found in: sub-001
eeg not found in: sub-002
eeg not found in: sub-003
eeg not found in: sub-004
eeg not found in: sub-005
eeg not found in: sub-006
eeg not found in: sub-007
eeg not found in: sub-008
eeg not found in: sub-009
G:\PRJ Roozbeh\science worker\projects\Parham Taherzade\dataset\derivatives\sub-010\eeg\sub-010_task-eyesclosed_eeg.set


<timed exec>:15: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
<timed exec>:15: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.


eeg not found in: sub-011
eeg not found in: sub-012
eeg not found in: sub-013
eeg not found in: sub-014
eeg not found in: sub-015
eeg not found in: sub-016
G:\PRJ Roozbeh\science worker\projects\Parham Taherzade\dataset\derivatives\sub-017\eeg\sub-017_task-eyesclosed_eeg.set


<timed exec>:15: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
<timed exec>:15: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.


eeg not found in: sub-018
G:\PRJ Roozbeh\science worker\projects\Parham Taherzade\dataset\derivatives\sub-019\eeg\sub-019_task-eyesclosed_eeg.set


...

eeg not found in: sub-076
eeg not found in: sub-077
eeg not found in: sub-078
eeg not found in: sub-079
eeg not found in: sub-080
eeg not found in: sub-081
eeg not found in: sub-082
eeg not found in: sub-083
eeg not found in: sub-084
eeg not found in: sub-085
G:\PRJ Roozbeh\science worker\projects\Parham Taherzade\dataset\derivatives\sub-086\eeg\sub-086_task-eyesclosed_eeg.set


<timed exec>:15: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
<timed exec>:15: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.


eeg not found in: sub-087
G:\PRJ Roozbeh\science worker\projects\Parham Taherzade\dataset\derivatives\sub-088\eeg\sub-088_task-eyesclosed_eeg.set


<timed exec>:15: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
<timed exec>:15: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.


CPU times: total: 4min 12s
Wall time: 4min 17s


In [129]:
result['sub-010']['O1']

[{'Delta_rel': 0.8068624572233134,
  'Theta_rel': 0.0379597314720536,
  'Alpha_rel': 0.1049070634163956,
  'Sigma_rel': 0.03025030632027808,
  'Beta_rel': 0.017545839069976553,
  'Gamma_rel': 0.002474602497983132},
 {'Delta_rel': 0.9228774780860741,
  'Theta_rel': 0.03693703953097986,
  'Alpha_rel': 0.020682592392176263,
  'Sigma_rel': 0.008004882378074044,
  'Beta_rel': 0.008324732352262925,
  'Gamma_rel': 0.003173275260432962},
 {'Delta_rel': 0.8635817733807756,
  'Theta_rel': 0.03731297635513926,
  'Alpha_rel': 0.054162389821277364,
  'Sigma_rel': 0.023300082170618748,
  'Beta_rel': 0.019116455783090486,
  'Gamma_rel': 0.002526322489098636},
 {'Delta_rel': 0.803352416070978,
  'Theta_rel': 0.09617392589454321,
  'Alpha_rel': 0.0637751421981898,
  'Sigma_rel': 0.01651932875862871,
  'Beta_rel': 0.01608019937911949,
  'Gamma_rel': 0.004098987698541127},
 {'Delta_rel': 0.9202084086196527,
  'Theta_rel': 0.04133402952498984,
  'Alpha_rel': 0.011566532657308887,
  'Sigma_rel': 0.00601193

In [77]:
joblib.dump(result, "../result/PT Powerband Features.joblib")

['../result/PT Powerband Features.joblib']

In [120]:
%%time
features = list()
feature_names = list()
labels = list()
groups = list()

for k in result.keys():
    names = list()
    feature = list()
    
    for ch in result[k].keys():
        df = pd.DataFrame(result[r][ch])
        names.extend(list(map(lambda x: f"{ch}_{x}", df.columns)))
        feature.append(df.to_numpy())

    feature = np.hstack(feature)
    groups.extend([k for _ in range(feature.shape[0])])
    labels.extend([participants[participants.participant_id == k].Group.values[0] for _ in range(feature.shape[0])])
    features.append(feature)
    
features_labels = pd.DataFrame(np.hstack([np.vstack(features), np.array(groups).reshape((-1, 1)), np.array(labels).reshape((-1, 1))]), columns=names+["group", "label"])

CPU times: total: 3.2 s
Wall time: 3.18 s


In [119]:
len(labels), len(groups), np.vstack(features).shape, len(names)

(7668, 7668, (7668, 114), 114)

In [121]:
features_labels

,Fp1_Delta_rel,Fp1_Theta_rel,Fp1_Alpha_rel,Fp1_Sigma_rel,Fp1_Beta_rel,Fp1_Gamma_rel,Fp2_Delta_rel,Fp2_Theta_rel,Fp2_Alpha_rel,Fp2_Sigma_rel,...,Cz_Beta_rel,Cz_Gamma_rel,Pz_Delta_rel,Pz_Theta_rel,Pz_Alpha_rel,Pz_Sigma_rel,Pz_Beta_rel,Pz_Gamma_rel,group,label
0,0.900606734536863,0.04286823151137524,0.034797454282247495,0.008630815274931848,0.009760065133189865,0.0033366992613924297,0.9080030039154056,0.04329789136184003,0.028488178672374798,0.007157913806297337,...,0.012003731237208978,0.0019014019169638085,0.8852114160603475,0.04142976321623347,0.03973583227092042,0.013207755822555668,0.01844067965535821,0.001974552974584572,sub-010,A
1,0.9402190692574207,0.033929216477584505,0.009834155106402908,0.005304330092543002,0.008431940379923761,0.002281288686124858,0.9400956592246967,0.032610307032861136,0.010225771627668774,0.004881958196250438,...,0.007802401652559294,0.0023658425683825523,0.9381834406943786,0.03697782645929268,0.012353621808621824,0.002955093987688942,0.006778214073271894,0.002751802976746265,sub-010,A
2,0.9176444954153478,0.044289488928530366,0.012606294369822225,0.010303407670402892,0.010753421413570056,0.004402892202326573,0.9190879192828683,0.045026395981986295,0.013218200251209766,0.009717546620415582,...,0.010943725382393613,0.002597509736119924,0.8888517082533635,0.045650680771009325,0.03733262651240658,0.014286964246109424,0.01167820563185632,0.0021998145852546486,sub-010,A
3,0.8548714363390606,0.09269297993632708,0.02185005569029526,0.00800493726892548,0.01501572821547565,0.007564862549915799,0.8716180871480672,0.0845671449901439,0.019762829006569015,0.006531979631812389,...,0.011096817016651514,0.004319964766194869,0.8503373324493023,0.10239912673214344,0.02180842391788095,0.009798871946706476,0.012031896388852945,0.0036243485651140258,sub-010,A
4,0.9114466677989057,0.04942847464294399,0.007042196951820351,0.00742402468407239,0.01499867559838786,0.009659960323869555,0.9124784152848706,0.052662445574403444,0.005839968822160841,0.007016999438835246,...,0.011809116261232768,0.004055617196902478,0.9152640766759795,0.051136706672064484,0.01171711280376429,0.004679828786718193,0.013737966774151978,0.003464308287321449,sub-010,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,0.887037543702781,0.0561036136951137,0.025187641991267894,0.01444333442479916,0.014040656646575517,0.003187209539462563,0.9007159164370082,0.050961359402413434,0.022023232617858355,0.010735681334332208,...,0.010139810433240644,0.0033238423672163403,0.91876119154918,0.04157335151555513,0.010255176431057668,0.013749861471830948,0.013062294533209022,0.0025981244991668093,sub-088,F
7664,0.8904171889523171,0.06754346913181948,0.013925671702920868,0.010713250658590847,0.013335127018394732,0.004065292535957185,0.8890536115700136,0.06421464102524632,0.01737520578799279,0.01214319590674026,...,0.012274626442957804,0.003853392137088895,0.9397163880245332,0.043268644141030765,0.007622418912186764,0.00441268612595792,0.0037225159032713974,0.0012573468930201475,sub-088,F
7665,0.965960950933119,0.01570854800184207,0.006429891685654057,0.0035362030951174854,0.006438713128486477,0.001925693155780842,0.9682190966352857,0.01598145509308638,0.005592320322186497,0.0033844027713236095,...,0.007048561340022909,0.0016506701886172918,0.859258445597042,0.11222160218076224,0.011619788243940441,0.008442001545483337,0.007077999628656496,0.001380162804115593,sub-088,F
7666,0.9013653780563299,0.04985797602680319,0.024740567453096184,0.008023382303670584,0.011052604712263527,0.004960091447836774,0.9209098663145773,0.03830813390495008,0.018172861188873995,0.009070178866744784,...,0.011228610034131777,0.0034279213999783317,0.9849730807044281,0.009311520420904003,0.003003324603820785,0.0015709718009023052,0.0009386763296757429,0.00020242614026885205,sub-088,F


In [122]:
joblib.dump(features_labels, "../result/PT dataset features label.joblib")

['../dataset/PT dataset features label.joblib']